In [1]:
pip install gurobipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Zielsetzung:
Es geht um die Terminplanung einer Physiopraxis. Es gillt die Arbeitszeiten der Angestellten so zu verteilen das die Räumlichkeiten optimal ausgenutzt werden.

Hard Constrain allgemein:
- Räume: 2 Behandlungsräume + 1 Kursraum
- Kursraum kann nur von einer Angestellten belegt sein
- Wenn der Kursraum belegt ist kann maximal ein zusätzlicher Raum belegt sein
- Ein Patient dauert 30min
- Arbeitszeit ist Mo-Fr 0800-2030

Soft Constrain allgemein:
- Wenn immer möglich sollen die Angestellten den Tag durch im gleichen Raum arbeiten können

Hard Constrain 1te Angestellte - Terapeut:
- Max 4h ohne Pause (240min)
- Min 1h Pause (60min)
- Während der Pause belegt die Angestellte keinen Raum
- Max Arbeitszeit Total 9.5h
- Max 4h pro Tag
- Eine Jogalektion belegt den Kursraum
- Jogalektion: Mo: 1500-1700 
- Ausserhalb der Jogalektion wird der Jogaraum nicht von der Angestellten belegt

Hard Constrain 2te Angestellte - Terapeut:
- Arbeitet Mo-Fr von 0800-1200
- Keine Pausen
- Darf den Joga Raum nicht belegen

Hard Constrain 3te Angestellte - Büroangestellte:
- Arbeitszeit Mo-Fr von 0800-2000
- Maximal 4h ohne Pause
- Mindestens 2h Arbeit ohne Pause
- Belegt einen Behandlungsraum
- Nicht mehr als 8.5h pro Tag

In [114]:
%%time
# Import librarys
import gurobipy as gp
from gurobipy import GRB

m = gp.Model()

# 08:00 - 20:30 je 30min Blöcke -> 24 Einheiten
t = 24

# 2 Patientenräume, 1 Jogaraum
r = 3
r0 = 0 # Patienten Raum 1
r1 = 1 # Patienten Raum 2
r2 = 2 # Jogaraum

# 3 Angestellte
e = 3
e0 = 0 # 1te Angestellte
e1 = 1 # 2te Angestellte
e2 = 2 # 3te Angestellte

d = m.addVars(t,e,r, vtype=GRB.BINARY)

t_dict = {0:"08:00",
          1:"08:30",
          2:"09:00",
          3:"09:30",
          4:"10:00",
          5:"10:30",
          6:"11:00",
          7:"11:30",
          8:"12:00",
          9:"12:30",
          10:"13:00",
          11:"13:30",
          12:"14:00",
          13:"14:30",
          14:"15:00",
          15:"15:30",
          16:"16:00",
          17:"16:30",
          18:"17:00",
          19:"17:30",
          20:"18:00",
          21:"18:30",
          22:"19:00",
          23:"19:30",
          24:"20:00"
          }

###################################################################################
# Helper
def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val][0]

###################################################################################
# Allgemein

## Hard Constrain: Je Zeiteinheit darf eine Angestellte nur in einem raum sein
for slot in range(t):
    for emp in range(e):
        m.addConstr(d[slot,emp,r0] + d[slot,emp,r1] + d[slot,emp,r2] <= 1)

## Hard Constrain: Je Zeiteinheit darf ein Raum nur einmal belegt sein
for slot in range(t):
    for room in range(r):
        m.addConstr(d[slot,e0,room] + d[slot,e1,room] + d[slot,e2,room] <= 1)

## Hard Constrain: Es können maximal zwei Angestellte gleichzeitig arbeiten
for slot in range(t):
    for room in range(r):
        m.addConstr(gp.quicksum(d[slot, emp, room] for emp in range(e)) <= 2)

## Soft Constrain: 
# Wenn immer möglich sollen die Angestellten den Tag durch im gleichen Raum arbeiten
# Kostenfunktion für Raumwechsel der Therapeuten
costRoomChange = 0
room_switch_cost = 10

for emp in range(e):
    for slot in range(t-3):
        # Wenn Therapeut in zwei aufeinanderfolgenden Zeitslots in unterschiedlichen Räumen arbeitet, erhöhe Kosten
        if gp.and_(d[slot, emp, r0], d[slot+1, emp, r1]):
            costRoomChange += room_switch_cost
        elif gp.and_(d[slot, emp, r1], d[slot+1, emp, r0]):
            costRoomChange += room_switch_cost


################################################################################## 
# 1te Angestellte

## Hard Constrain: Arbeitet zwischen 1500-1700 in Jogaraum
offset = get_keys_from_value(t_dict, "15:00")
for slot in range(4):
    m.addConstr((d[offset + slot, e0, r2] == 1))

## Hard Constrain Ausserhalb der Jogalektion wird der Jogaraum nicht von der Angestellten belegt
for slot in range(offset):
    m.addConstr((d[slot, e0, r2] == 0))

for slot in range(offset+4, t):
    m.addConstr((d[slot, e0, r2] == 0))

## Hard Constraint: Max 4h ohne Pause (240min)
# Je timeslot prüfen das nicht länger als 8 aufeinander folgende arbeiten getätigt werden
for slot in range(t-7): # Alle timeslot bis auf 3.5h vor ende
    expr = 0
    const = False
    for i in range(9):
        if slot+i < t:
            expr += d[slot+i,e0,r0] + d[slot+i,e0,r1] + d[slot+i,e0,r2]
            constr = True
    if constr:
        m.addConstr(expr <= 8) # Maximale dauer ohne Pause 8 -> 4h

## Hard Constraint: Max Arbeitszeit Total 9:30h (570min) -> 19
m.addConstr(gp.quicksum(d[slot, e0, r0]+d[slot, e0, r1]+d[slot, e0, r2] for slot in range(t)) <= 19)

## Hard Constraint: Minimale Wochenarbeitszeit am Patienten 20h (1200min) -> min 4h pro Tag
m.addConstr(gp.quicksum(d[slot, e0, r0]+d[slot, e0, r1]+d[slot, e0, r2] for slot in range(t)) >= 8)

## Hard Constraint: Min 1h Pause am Stück (60min)
# for slot in range(t-2):
#     # Prüfe ob eine Pause auftritt
#     a = d[slot,e0,r0] + d[slot,e0,r1] + d[slot,e0,r2]
#     # Wenn eine Pause auftritt dann soll diese mind 60min dauern:
#     b = (d[slot+1,e0,r0] + d[slot+1,e0,r1] + d[slot+1,e0,r2])
#     m.addConstr((a==0) >> b==0)

#############################################################################
# 2te Angestellte
 
## Hard Constrain: Arbeitet von 0800-1200 und besetzt dabei einen Behandlungsraum
# 0 - 7 -- 0800-1200 Arbeiten
for slot in range(8):
    m.addConstr(d[slot, e1, r0] + d[slot, e1, r1] == 1)
# Den rest nicht
for slot in range(9,t):
    m.addConstr(d[slot, e1, r0] + d[slot, e1, r1] == 0)

## Hard Constrain: Darf den Jogaraum nicht belegen
for slot in range(t):
    m.addConstr(d[slot, e1, r2] == 0)

## Hard Constrain: keine Pausen
# - > Ist bereits gegeben mit dem obigen Constrain

#############################################################################
# 3te Angestellte

## Hard Constrain: Arbeitszeit Mo-Fr 0800-2000
# Gegeben durch timeslots

## Hard Constrain: Max 4h ohne Pause (240min)
# Je timeslot prüfen das nicht länger als 8 aufeinander folgende arbeiten getätigt werden
for slot in range(t-7): # Alle timeslot bis auf 3.5h vor ende
    expr = 0
    const = False
    for i in range(9):
        if slot+i < t:
            expr += d[slot+i,e2,r0] + d[slot+i,e2,r1]
            constr = True
    if constr:
        m.addConstr(expr <= 8) # Maximale dauer ohne Pause 8 -> 4h
            
## Hard Constrain: Darf den Jogaraum nicht belegen
for slot in range(t):
    m.addConstr(d[slot, e2, r2] == 0)

## Hard Constrain: Die dritte Angestellte sollte nicht mehr als 8.5h Arbeiten
m.addConstr(gp.quicksum(d[slot, e2, r0]+d[slot, e2, r1] for slot in range(t)) <= 17)

## Hard Constrain: Min 2h Arbeiten am Stück ohne Pause
#for slot in range(t-8):
    #expr = 0
    # Verhindern das wenn die maximale Arbeitszeit erreicht ist die Regel aktiv ist
    #if gp.or_(d[slot,e2,r0] + d[slot,e2,r1]):
    #    for i in range(5):
    #        expr += d[slot+i,e2,r0] + d[slot+i,e2,r1]
    #    m.addConstr(expr >= 4)

############################################################################
# Zielfuntkion

einkommen = 0
for slot in range(t):
    for emp in range(e):
        einkommen += (d[slot,emp,r0] + d[slot,emp,r1] + d[slot,emp,r2]) * 50

m.setObjective(einkommen - costRoomChange, GRB.MAXIMIZE)
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: 12th Gen Intel(R) Core(TM) i9-12900H, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 348 rows, 216 columns and 1718 nonzeros
Model fingerprint: 0x4a4f887d
Variable types: 0 continuous, 216 integer (216 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 1520.0000000
Presolve removed 259 rows and 146 columns
Presolve time: 0.00s
Presolved: 89 rows, 70 columns, 579 nonzeros
Found heuristic solution: objective 1520.0000000
Variable types: 0 continuous, 70 integer (70 binary)

Root relaxation: objective 1.620000e+03, 25 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Be

In [108]:
print("Räume")
print("TIME \t R1: A1 A2 A3\t\tR2: A1 A2 A3\t\tR3: A1 A2 A3")
for slot in range(t):
    print(f" {t_dict[slot]} \t R1: {int(d[slot, e0, r0].x)}  {int(d[slot, e1, r0].x)}  {int(d[slot, e2, r0].x)}\
          \tR2: {int(d[slot, e0, r1].x)}  {int(d[slot, e1, r1].x)}  {int(d[slot, e2, r1].x)}\
          \tR3: {int(d[slot, e0, r2].x)}  {int(d[slot, e1, r2].x)}  {int(d[slot, e2, r2].x)} ")


Räume
TIME 	 R1: A1 A2 A3		R2: A1 A2 A3		R3: A1 A2 A3
 08:00 	 R1: 0  1  0          	R2: 0  0  1          	R3: 0  0  0 
 08:30 	 R1: 0  1  0          	R2: 0  0  1          	R3: 0  0  0 
 09:00 	 R1: 0  1  0          	R2: 0  0  1          	R3: 0  0  0 
 09:30 	 R1: 1  0  0          	R2: 0  1  0          	R3: 0  0  0 
 10:00 	 R1: 1  0  0          	R2: 0  1  0          	R3: 0  0  0 
 10:30 	 R1: 0  1  0          	R2: 1  0  0          	R3: 0  0  0 
 11:00 	 R1: 1  0  0          	R2: 0  1  0          	R3: 0  0  0 
 11:30 	 R1: 0  1  0          	R2: 1  0  0          	R3: 0  0  0 
 12:00 	 R1: 1  0  0          	R2: 0  1  0          	R3: 0  0  0 
 12:30 	 R1: 0  0  1          	R2: 0  0  0          	R3: 0  0  0 
 13:00 	 R1: 0  0  1          	R2: 1  0  0          	R3: 0  0  0 
 13:30 	 R1: 0  0  1          	R2: 1  0  0          	R3: 0  0  0 
 14:00 	 R1: 0  0  1          	R2: 1  0  0          	R3: 0  0  0 
 14:30 	 R1: 0  0  1          	R2: 1  0  0          	R3: 0  0  0 
 15:00 	 R1: 0  0  0  

In [109]:
print("Angestellte")
print(" TIME \t A1: R1 R2 R3\t\tA2: R1 R2 R3\t\tA3: R1 R2 R3")
for slot in range(t):
    print(f" {t_dict[slot]} \t A1: {int(d[slot, e0, r0].x)}  {int(d[slot, e0, r1].x)}  {int(d[slot, e0, r2].x)}\
          \tA2: {int(d[slot, e1, r0].x)}  {int(d[slot, e1, r1].x)}  {int(d[slot, e1, r2].x)}\
          \tA3: {int(d[slot, e2, r0].x)}  {int(d[slot, e2, r1].x)}  {int(d[slot, e2, r2].x)}")

Angestellte
 TIME 	 A1: R1 R2 R3		A2: R1 R2 R3		A3: R1 R2 R3
 08:00 	 A1: 0  0  0          	A2: 1  0  0          	A3: 0  1  0
 08:30 	 A1: 0  0  0          	A2: 1  0  0          	A3: 0  1  0
 09:00 	 A1: 0  0  0          	A2: 1  0  0          	A3: 0  1  0
 09:30 	 A1: 1  0  0          	A2: 0  1  0          	A3: 0  0  0
 10:00 	 A1: 1  0  0          	A2: 0  1  0          	A3: 0  0  0
 10:30 	 A1: 0  1  0          	A2: 1  0  0          	A3: 0  0  0
 11:00 	 A1: 1  0  0          	A2: 0  1  0          	A3: 0  0  0
 11:30 	 A1: 0  1  0          	A2: 1  0  0          	A3: 0  0  0
 12:00 	 A1: 1  0  0          	A2: 0  1  0          	A3: 0  0  0
 12:30 	 A1: 0  0  0          	A2: 0  0  0          	A3: 1  0  0
 13:00 	 A1: 0  1  0          	A2: 0  0  0          	A3: 1  0  0
 13:30 	 A1: 0  1  0          	A2: 0  0  0          	A3: 1  0  0
 14:00 	 A1: 0  1  0          	A2: 0  0  0          	A3: 1  0  0
 14:30 	 A1: 0  1  0          	A2: 0  0  0          	A3: 1  0  0
 15:00 	 A1: 0  0  1         

In [117]:
sum  = 0
for slot in range(t):
    for emp in range(e):
        sum += int(d[slot, emp, r0].x) + int(d[slot, emp, r1].x) + int(d[slot, emp, r2].x)
sum = sum/2
print(f"Total geleistete Arbeitsstunden: {sum}h")

Total geleistete Arbeitsstunden: 22.5h
